In [1]:
# This is necessary to recognize the modules
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

import asyncio
from decimal import Decimal
from core.data_sources import CLOBDataSource
from core.features.candles.volatility import VolatilityConfig
from research_notebooks.xtreet_bb.utils import generate_config, dump_dict_to_yaml

clob = CLOBDataSource()

In [2]:
# Screener parameters
CONNECTOR_NAME = "binance_perpetual"
INTERVALS = ["1m"]
DAYS = 7
FETCH_CANDLES = True
BATCH_CANDLES_REQUEST = 15
SLEEP_REQUEST = 60.0
VOLUME_THRESHOLD = 0.3  # From percentile VOLUME_THRESHOLD to 1
VOLATILITY_THRESHOLD = 0.4  # From percentile VOLATILITY_THRESHOLD to 1

# Trading Rules Filter
QUOTE_ASSET = "USDT"
MIN_NOTIONAL_SIZE = 5  # In USDT
MAX_PRICE_STEP = 0.001  # Min price step in % (tick size)

VOLATILITY_WINDOW = 60  # In bars
VOLUME_FAST_WINDOW = 20  # No se usa
VOLUME_SLOW_WINDOW = 100  # No se usa

# Config generation
TOTAL_AMOUNT = 300  # General total amount for all markets
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_EXECUTORS_PER_SIDE = 1  # Maximum number of executors per side
COOLDOWN_TIME = 0
LEVERAGE = 20 # Should be for each trading pair
TIME_LIMIT = 60 * 60 * 24
BOLLINGER_LENGTHS = [50, 100, 200]
BOLLINGER_STDS = [1.4, 1.8, 2.2]
SL_STD_MULTIPLIER = 2
TS_DELTA_MULTIPLIER = 0.2

# Config filtering
MIN_DISTANCE_BETWEEN_ORDERS = 0.01
MAX_TS_SL_RATIO = 0.5

# Download data
- Get trading rules
- Get candles for the last x days

In [3]:
trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset(QUOTE_ASSET)\
    .filter_by_min_notional_size(Decimal(MIN_NOTIONAL_SIZE))\
    .get_all_trading_pairs()

if FETCH_CANDLES:
    number_of_calls = (len(trading_pairs) // BATCH_CANDLES_REQUEST) + 1

    all_candles = {}

    for i in range(number_of_calls):
        print(f"Batch {i + 1}/{number_of_calls}")
        start = i * BATCH_CANDLES_REQUEST
        end = (i + 1) * BATCH_CANDLES_REQUEST
        print(f"Start: {start}, End: {end}")
        end = min(end, len(trading_pairs))
        trading_pairs_batch = trading_pairs[start:end]

        tasks = [clob.get_candles_last_days(
        connector_name=CONNECTOR_NAME,
        trading_pair=trading_pair,
        interval=interval,
        days=DAYS,
        ) for trading_pair in trading_pairs_batch for interval in INTERVALS]

        candles = await asyncio.gather(*tasks)
        candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}
        all_candles.update(candles)
        if i != number_of_calls - 1:
            print(f"Sleeping for {SLEEP_REQUEST} seconds")
            await asyncio.sleep(SLEEP_REQUEST)
    clob.dump_candles_cache(os.path.join(root_path, "data"))
else:
    clob.load_candles_cache(os.path.join(root_path, "data"))
candles = [value for key, value in clob.candles_cache.items() if key[2] in INTERVALS and key[0] == CONNECTOR_NAME]

2024-08-31 00:00:33,830 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x151826ec0>
2024-08-31 00:00:33,831 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1068f1a80>, 80771.015283041)]']
connector: <aiohttp.connector.TCPConnector object at 0x151826e90>


Batch 1/19
Start: 0, End: 15


2024-08-31 00:00:40,053 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x15189a7d0>
2024-08-31 00:00:40,054 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1517b1ae0>, 80777.225385458)]']
connector: <aiohttp.connector.TCPConnector object at 0x15189a800>
2024-08-31 00:00:40,065 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x15185dcc0>
2024-08-31 00:00:40,066 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1517b0d00>, 80777.24102825)]']
connector: <aiohttp.connector.TCPConnector object at 0x15185dcf0>
2024-08-31 00:00:40,077 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1518987c0>
2024-08-31 00:00:40,078 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1517

Sleeping for 60.0 seconds
Batch 2/19
Start: 15, End: 30


2024-08-31 00:01:44,799 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x151917940>
2024-08-31 00:01:44,800 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1517b00a0>, 80841.975423541)]']
connector: <aiohttp.connector.TCPConnector object at 0x1519142e0>
2024-08-31 00:01:44,814 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x15185d4b0>
2024-08-31 00:01:44,815 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151954be0>, 80841.991225791)]']
connector: <aiohttp.connector.TCPConnector object at 0x15185c4c0>
2024-08-31 00:01:44,827 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1518d40d0>
2024-08-31 00:01:44,828 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151

...

2024-08-31 00:17:53,769 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x151d5c040>
2024-08-31 00:17:53,770 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151b43ee0>, 81810.932111875)]']
connector: <aiohttp.connector.TCPConnector object at 0x151d5c1c0>
2024-08-31 00:17:53,933 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x151a3ef20>
2024-08-31 00:17:53,933 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151b5d720>, 81811.10198975)]']
connector: <aiohttp.connector.TCPConnector object at 0x151a3f2b0>
2024-08-31 00:17:55,424 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1518d79d0>
2024-08-31 00:17:55,425 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151b

Sleeping for 60.0 seconds
Batch 18/19
Start: 255, End: 270


2024-08-31 00:18:59,873 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x151a1d600>
2024-08-31 00:18:59,874 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151d342e0>, 81877.040869541)]']
connector: <aiohttp.connector.TCPConnector object at 0x151a1c640>
2024-08-31 00:19:00,039 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x151d5c3d0>
2024-08-31 00:19:00,040 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151b5ee60>, 81877.201234583)]']
connector: <aiohttp.connector.TCPConnector object at 0x151d5d570>
2024-08-31 00:19:00,061 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x151a3d2a0>
2024-08-31 00:19:00,061 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151

Sleeping for 60.0 seconds
Batch 19/19
Start: 270, End: 285


2024-08-31 00:20:04,296 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x151a7d4b0>
2024-08-31 00:20:04,297 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x151956260>, 81941.456991625)]']
connector: <aiohttp.connector.TCPConnector object at 0x151a7fbb0>


In [11]:
from core.features.candles.volume import VolumeConfig
from research_notebooks.xtreet_bb.utils import generate_screener_report

screner_report = generate_screener_report(
    candles=candles,
    trading_rules=trading_rules,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_FAST_WINDOW, long_window=VOLUME_FAST_WINDOW))
screner_report["url"] = screner_report["trading_pair"].apply(lambda x: f"https://www.binance.com/en/trade/{x}")
screner_report.sort_values("mean_natr", ascending=False, inplace=True)
screner_report

,trading_pair,mean_natr,average_volume_per_hour,price_step_pct,url
30,XEM-USDT,0.00559602,846412.32974442,0.00558659,https://www.binance.com/en/trade/XEM-USDT
254,DOGS-USDT,0.00557404,58372508.26445325,0.00007608,https://www.binance.com/en/trade/DOGS-USDT
53,VIDT-USDT,0.00483275,9531744.99586725,0.00019889,https://www.binance.com/en/trade/VIDT-USDT
5,SUN-USDT,0.00480054,29867774.9081979,0.00002775,https://www.binance.com/en/trade/SUN-USDT
179,MBOX-USDT,0.00447024,3666078.91249911,0.00060132,https://www.binance.com/en/trade/MBOX-USDT
...,...,...,...,...,...
155,LOOM-USDT,0.00079939,192434.0459994,0.00022381,https://www.binance.com/en/trade/LOOM-USDT
265,XLM-USDT,0.00076637,1482130.11107517,0.00010727,https://www.binance.com/en/trade/XLM-USDT
55,TRX-USDT,0.00074257,8568631.95350011,0.00006275,https://www.binance.com/en/trade/TRX-USDT
99,BTCDOM-USDT,0.00038206,164958.52376029,0.0000354,https://www.binance.com/en/trade/BTCDOM-USDT


In [13]:
# Calculate the 20th percentile (0.2 quantile) for both columns
natr_percentile = screner_report['mean_natr'].quantile(VOLATILITY_THRESHOLD)
volume_percentile = screner_report['average_volume_per_hour'].quantile(VOLUME_THRESHOLD)

# Filter the DataFrame to get observations where mean_natr is greater than its 20th percentile
# and average_volume_per_hour is greater than its 20th percentile
screener_top_markets = screner_report[
    (screner_report['mean_natr'] > natr_percentile) &
    (screner_report['average_volume_per_hour'] > volume_percentile) &
    (screner_report["price_step_pct"] < MAX_PRICE_STEP)
]

# Display the filtered DataFrame
screener_top_markets[["trading_pair", "mean_natr", "average_volume_per_hour", "price_step_pct", "url"]]

,trading_pair,mean_natr,average_volume_per_hour,price_step_pct,url
254,DOGS-USDT,0.00557404,58372508.26445325,0.00007608,https://www.binance.com/en/trade/DOGS-USDT
53,VIDT-USDT,0.00483275,9531744.99586725,0.00019889,https://www.binance.com/en/trade/VIDT-USDT
5,SUN-USDT,0.00480054,29867774.9081979,0.00002775,https://www.binance.com/en/trade/SUN-USDT
179,MBOX-USDT,0.00447024,3666078.91249911,0.00060132,https://www.binance.com/en/trade/MBOX-USDT
240,CHESS-USDT,0.0044066,3825026.63651922,0.00054885,https://www.binance.com/en/trade/CHESS-USDT
...,...,...,...,...,...
51,COTI-USDT,0.00161843,384330.13129969,0.00012189,https://www.binance.com/en/trade/COTI-USDT
38,AAVE-USDT,0.0016094,5634949.37285643,0.00007622,https://www.binance.com/en/trade/AAVE-USDT
234,ZEC-USDT,0.00160707,1461995.75748517,0.00030451,https://www.binance.com/en/trade/ZEC-USDT
242,BOND-USDT,0.00160299,557097.81297887,0.00050025,https://www.binance.com/en/trade/BOND-USDT


In [14]:
strategy_configs = generate_config(
    connector_name=CONNECTOR_NAME,
    intervals=INTERVALS,
    screener_top_markets=screener_top_markets.head(10),
    candles=candles,
    total_amount=TOTAL_AMOUNT,
    max_executors_per_side=MAX_EXECUTORS_PER_SIDE,
    cooldown_time=COOLDOWN_TIME,
    leverage=LEVERAGE,
    time_limit=TIME_LIMIT,
    bb_lengths=BOLLINGER_LENGTHS,
    bb_stds=BOLLINGER_STDS,
    sl_std_multiplier=SL_STD_MULTIPLIER,
    min_distance_between_orders=MIN_DISTANCE_BETWEEN_ORDERS,
    max_ts_sl_ratio=MAX_TS_SL_RATIO,
    ts_delta_multiplier=TS_DELTA_MULTIPLIER,
)

Skipping MBOX-USDT due to stop loss closer to get executed:BEP: 0.006787327997462844, SL: 0.005650393703021291, S1: 0.012935499940133044
Skipping MBOX-USDT due to stop loss closer to get executed:BEP: 0.009389684794780528, SL: 0.005316370632364347, S1: 0.01805967089332271
Skipping ALPACA-USDT due to stop loss closer to get executed:BEP: 0.00750511475971094, SL: 0.006146736229079226, S1: 0.014312286325927258
Skipping ALPACA-USDT due to stop loss closer to get executed:BEP: 0.016362533227223697, SL: 0.010508155583437954, S1: 0.027522119959094753
Skipping MBOX-USDT due to stop loss closer to get executed:BEP: 0.03248710201186622, SL: 0.01661545801176096, S1: 0.05424904491331503
Skipping ALPACA-USDT due to stop loss closer to get executed:BEP: 0.015188700578061543, SL: 0.007799671592775008, S1: 0.024812226686423872
Skipping NULS-USDT due to stop loss closer to get executed:BEP: 0.015343187239204914, SL: 0.010876256925918381, S1: 0.027064117889066937
Skipping NULS-USDT due to stop loss clos

In [15]:
len(set([config["trading_pair"] for config in strategy_configs]))

10

In [16]:
len(strategy_configs)

462

In [17]:

# DCA amounts
for config in strategy_configs:
    dump_dict_to_yaml("configs/", config)